<a href="https://colab.research.google.com/github/1021114Carlos/risk-free-assets-to-generate-income/blob/main/risk_free_assets_to_generate_income.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Library

In [ ]:
import random
import calendar
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
# What is the future value of $1 invested in a Share Certificate/Certificate of deposit that matures in 3 months?
# annual percentage rate 3.5% and effective annual rate 3.56%

# Verify effective annual rate is compounded monthly
ear = ((1 + (0.0350/12))**(12)) - 1
f'Effective annual rate is compounded monthly and it is {ear:.6f} or {(ear*100):.2f}%'

'Effective annual rate is compounded monthly and it is 0.035567 or 3.56%'

In [ ]:
  # Investing on a share certificate with 3 month maturity.
amount3Month = float(input("Amount to save: "))

rate = float(input('Annual percentage yield: '))
maturity = float(input("Time Horizon: "))
time = maturity/12
ear = rate/100

shareCertificate3Month = amount3Month*(1 + ear)**(time)
f'If investing $ {amount3Month} in a 3 month share certificate, the return would be ${shareCertificate3Month:.3f}'

Amount to save: 100
Annual percentage yield: 3.5
Time Horizon: 3


'If investing $ 100.0 in a 3 month share certificate, the return would be $100.864'

In [ ]:
# Investing on a share certificate with 3 month duration.
amount6 = float(input("Amount to save: "))

rate = float(input('Annual percentage yield: '))
maturity = float(input("Term Duration: "))
time = maturity/12
ear = rate/100

shareCertificate6 = amount6*(1 + ear)**(time)
shareCertificate6
f'If investing 1k in a  month share certificate, the return would be ${shareCertificate6:.3f}'

KeyboardInterrupt: Interrupted by user

In [ ]:
total_3_6 = (shareCertificate3 - amount3) + (shareCertificate6 - amount6)
total_3_6

26.427932227021756

In [ ]:
# Investing on a share certificate with 3 month duration.
amountH6 = float(input("Amount to save: "))

rate = float(input('Annual percentage yield: '))
maturity = float(input("Term Duration: "))
time = maturity/12
ear = rate/100

shareCertificateH6 = amountH6*(1 + ear)**(time)

f'If investing 2k in a  month share certificate, the return would be ${shareCertificateH6:.3f}'

Amount to save: 2000
Annual percentage yield: 3.56
Term Duration: 6


'If investing 2k in a  month share certificate, the return would be $2035.289'

# Increasing personal purchasing power

In [ ]:
# How to increase personal purchasing power. it is true that to achieve this, we have to increase income, and make better spending decisions.
# How can we increase the purchasing power when we can't increase the income?

# Simply adding a % to remaining balance and adding to next month balance.
addToBalanceRemaining = 0
month = list(calendar.month_name)
for period in month[1:]:
  ppp = 1000
  monthlyExpense = random.randint(700, ppp)
  balanceRemaining = ppp - monthlyExpense
  addToBalanceRemaining += balanceRemaining*0.05
  replenishMonthlyExpense = (monthlyExpense + balanceRemaining) + addToBalanceRemaining
  print(f'{period:>10}  ${replenishMonthlyExpense:.2f}')

   January  $1001.15
  February  $1014.65
     March  $1019.75
     April  $1031.55
       May  $1034.60
      June  $1035.60
      July  $1040.85
    August  $1054.25
 September  $1059.55
   October  $1062.75
  November  $1075.95
  December  $1079.60


In [ ]:
# bigger balance every month. more expenses.
addToBalanceRemaining = 0
month = list(calendar.month_name)[1:]
for period in month:
  ppp = 1000 + int(addToBalanceRemaining)
  monthlyExpense = random.randint(700, ppp)
  balanceRemaining = ppp - monthlyExpense
  addToBalanceRemaining += balanceRemaining*0.05
  replenishMonthlyBudget = (monthlyExpense + balanceRemaining) + addToBalanceRemaining
  print(f'expensens for the month of {period:>10} is ${monthlyExpense:.2f} and replenishing the budget comes to ${replenishMonthlyBudget:.2f}')

expensens for the month of    January is $967.00 and replenishing the budget comes to $1001.65
expensens for the month of   February is $921.00 and replenishing the budget comes to $1006.65
expensens for the month of      March is $845.00 and replenishing the budget comes to $1018.65
expensens for the month of      April is $780.00 and replenishing the budget comes to $1038.30
expensens for the month of        May is $789.00 and replenishing the budget comes to $1062.10
expensens for the month of       June is $820.00 and replenishing the budget comes to $1084.95
expensens for the month of       July is $982.00 and replenishing the budget comes to $1098.20
expensens for the month of     August is $741.00 and replenishing the budget comes to $1117.70
expensens for the month of  September is $839.00 and replenishing the budget comes to $1144.05
expensens for the month of    October is $956.00 and replenishing the budget comes to $1162.15
expensens for the month of   November is $727.00 a

In [ ]:
addToBalanceRemaining = 0
month = list(calendar.month_name)[1:]
expenses = []
ppps = []
replenishing_budgets = []

for period in month:
    ppp = 1000 + int(addToBalanceRemaining)
    monthlyExpense = random.randint(700, ppp)
    balanceRemaining = ppp - monthlyExpense
    addToBalanceRemaining += balanceRemaining * 0.05
    replenishMonthlyBudget = (monthlyExpense + balanceRemaining) + addToBalanceRemaining
    print(f'expensens in {period:<10} is ${monthlyExpense:.2f} and replenishing the budget comes to ${replenishMonthlyBudget:.2f}')
    expenses.append(monthlyExpense)
    ppps.append(ppp)
    replenishing_budgets.append(replenishMonthlyBudget)

# Create interactive plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=month, y=expenses, mode='lines+markers', name='Expenses', line=dict(color='red')))
fig.add_trace(go.Scatter(x=month, y=ppps, mode='lines+markers', name='PPP', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=month, y=replenishing_budgets, mode='lines+markers', name='Replenishing Budget'))

fig.update_layout(
    title='Monthly Financial Overview',
    xaxis_title='Month',
    yaxis_title='Amount (USD)',
    hovermode='x unified'
)

fig.show()

expensens in January    is $831.00 and replenishing the budget comes to $1008.45
expensens in February   is $742.00 and replenishing the budget comes to $1029.75
expensens in March      is $996.00 and replenishing the budget comes to $1044.00
expensens in April      is $880.00 and replenishing the budget comes to $1053.15
expensens in May        is $808.00 and replenishing the budget comes to $1071.25
expensens in June       is $885.00 and replenishing the budget comes to $1090.05
expensens in July       is $731.00 and replenishing the budget comes to $1113.95
expensens in August     is $1039.00 and replenishing the budget comes to $1130.20
expensens in September  is $905.00 and replenishing the budget comes to $1140.25
expensens in October    is $752.00 and replenishing the budget comes to $1164.35
expensens in November   is $1009.00 and replenishing the budget comes to $1184.40
expensens in December   is $709.00 and replenishing the budget comes to $1207.65


In [ ]:
import calendar
import random
import pandas as pd
import plotly.graph_objects as go

addToBalanceRemaining = 0
months = list(calendar.month_name)[1:]
expenses = []
ppps = []
replenishing_budgets = []

for i, period in enumerate(months):
    if i == 0:
        ppp = 1000 + int(addToBalanceRemaining)
        monthlyExpense = random.randint(700, ppp)
        balanceRemaining = ppp - monthlyExpense

        print(f'expensens in {period:<10} is ${monthlyExpense:.2f} — replenishing budget is not calculated this month')

        expenses.append(monthlyExpense)
        ppps.append(ppp)
        replenishing_budgets.append(None)

        # Only track addToBalanceRemaining for future use
        addToBalanceRemaining += balanceRemaining * 0.05

    else:
        ppp = 1000
        monthlyExpense = random.randint(700, ppp)
        balanceRemaining = ppp - monthlyExpense
        addToBalanceRemaining += balanceRemaining * 0.05
        replenishMonthlyBudget = (monthlyExpense + balanceRemaining) + addToBalanceRemaining

        print(f'expensens in {period:<10} is ${monthlyExpense:.2f} and replenishing the budget comes to ${replenishMonthlyBudget:.2f}')

        expenses.append(monthlyExpense)
        ppps.append(ppp)
        replenishing_budgets.append(replenishMonthlyBudget)


# Plotly visualization
fig = go.Figure()
fig.add_trace(go.Scatter(x=months, y=ppps, mode='lines+markers', name='PPP'))
fig.add_trace(go.Scatter(x=months, y=expenses, mode='lines+markers', name='Expenses', line=dict(color='red')))
fig.add_trace(go.Scatter(x=months, y=replenishing_budgets, mode='lines+markers', name='Replenishing Budget'))

fig.update_layout(
    title='Monthly Financial Simulation',
    xaxis_title='Month',
    yaxis_title='Amount (USD)',
    hovermode='x unified'
)

fig.show()

expensens in January    is $876.00 — replenishing budget is not calculated this month
expensens in February   is $889.00 and replenishing the budget comes to $1011.75
expensens in March      is $754.00 and replenishing the budget comes to $1024.05
expensens in April      is $922.00 and replenishing the budget comes to $1027.95
expensens in May        is $949.00 and replenishing the budget comes to $1030.50
expensens in June       is $808.00 and replenishing the budget comes to $1040.10
expensens in July       is $978.00 and replenishing the budget comes to $1041.20
expensens in August     is $972.00 and replenishing the budget comes to $1042.60
expensens in September  is $757.00 and replenishing the budget comes to $1054.75
expensens in October    is $770.00 and replenishing the budget comes to $1066.25
expensens in November   is $823.00 and replenishing the budget comes to $1075.10
expensens in December   is $858.00 and replenishing the budget comes to $1082.20


First let's try to calculate return on us t bond

In [ ]:
bond_profit = (1000/99.14231)
bond_profit

1008.6510996162991